In [478]:
# !pip install matplotlib

In [479]:
# !pip install altair

In [480]:
# !pip install altair_saver

In [481]:
# !npm install vega-lite vega-cli canvas -g

In [482]:
# !pip install jupyterlab_code_formatter

In [483]:
# !pip install black

In [484]:
import json
import altair as alt
from altair import expr, datum
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import requests
import re

In [485]:
import colorsys
from matplotlib.colors import to_hex, to_rgb


def scale_lightness(rgb, scale_l):
    rgbhex = False
    if "#" in rgb:
        rgb = to_rgb(rgb)
        rgbhex = True
    # convert rgb to hls
    h, l, s = colorsys.rgb_to_hls(*rgb)
    # manipulate h, l, s values and return as rgb
    c = colorsys.hls_to_rgb(h, min(1, l * scale_l), s=s)
    if rgbhex:
        c = to_hex(c)
    return c

In [486]:
LOCAL = False

if LOCAL:
    local_suffix = "_local"
else:
    local_suffix = ""

In [487]:
%%capture pwd
!pwd

In [488]:
uid = pwd.stdout.split("/")[-1].split("\r")[0]
eco_git_home = (
    "https://raw.githubusercontent.com/EconomicsObservatory/ECOvisualisations/main/"
)
eco_git_path = eco_git_home + "articles/" + uid + "/data/"
vega_embed = requests.get(eco_git_home + "guidelines/html/vega-embed.html").text
colors = json.loads(
    requests.get(eco_git_home + "guidelines/colors/eco-colors.json").content
)
category_color = json.loads(
    requests.get(eco_git_home + "guidelines/colors/eco-category-color.json").content
)
hue_color = json.loads(
    requests.get(eco_git_home + "guidelines/colors/eco-single-hue-color.json").content
)
mhue_color = json.loads(
    requests.get(eco_git_home + "guidelines/colors/eco-multi-hue-color.json").content
)
div_color = json.loads(
    requests.get(eco_git_home + "guidelines/colors/eco-diverging-color.json").content
)
config = json.loads(
    requests.get(eco_git_home + "guidelines/charts/eco-global-config.json").content
)
height = config["height"]
width = config["width"]
uid, height, width

('how-are-agricultural-economies-affected-by-climate-change', 300, 500)

# Fig 1

In [489]:
df = pd.read_excel("raw/Book2.xlsx").dropna(axis=1, how="all")
dfs = []
for c in df.columns:
    dx = df[[c]]
    dx["x"] = dx[c].str.split(",").str[0]
    dx["y"] = dx[c].str.split(",").str[1]
    dx["year"] = c
    dx = dx.drop(c, axis=1)
    dx["x"] = dx["x"].astype(float).round(0)
    dfs.append(dx)
df = pd.concat(dfs).dropna()
df = (
    df.set_index(["x", "year"]).unstack()["y"].reset_index().astype(float).interpolate()
)
df["bin"] = 3.5 + df["bin"]
df["x"] = (
    ["< 6"] + [str(i * 3) + "-" + str((i + 1) * 3) for i in range(2, 14)] + ["42 <"]
)
df['z']=0

<ipython-input-489-ec1396431916>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dx["x"] = dx[c].str.split(",").str[0]
<ipython-input-489-ec1396431916>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dx["y"] = dx[c].str.split(",").str[1]


In [490]:
f = "fig1_productivity"
f1 = eco_git_path + f + ".csv"
df.to_csv("data/" + f + ".csv")
f += local_suffix
open("visualisation/" + f + ".html", "w").write(
    vega_embed.replace(
        "JSON_PATH", f1.replace("/data/", "/visualisation/").replace(".csv", ".json")
    )
)
if LOCAL:
    f1 = df
readme = "### " + f + '\n!["' + f + '"](visualisation/' + f + '.png "' + f + '")\n\n'
df.head()

year,x,a,bin,bottom,top,z
0,< 6,-1.234847,0.298799,-2.359851,-0.123322,0
1,6-9,-1.059189,0.393389,-1.907979,-0.230604,0
2,9-12,-0.236826,0.588671,-0.957616,0.504203,0
3,12-15,-0.586593,0.763804,-1.300646,0.080388,0
4,15-18,-0.808390,0.918829,-1.512335,-0.097652,0


In [491]:
base = alt.Chart(f1).encode(
    x=alt.X(
        "x:N",
        sort=[],
        axis=alt.Axis(
            grid=False,
            titleAlign="right",
            titleAnchor="end",
            titleY=-15,
            title="Temperature bin (°C)",
            labelColor=colors["eco-gray"],
            titleColor=colors["eco-gray"],
            tickColor=colors["eco-gray"],
            domainColor=colors["eco-gray"],
            tickCount=7,
            orient="bottom",
            labelAngle=0,
        ),
        # scale=alt.Scale(domain=[1, 14], nice=False),
    )
)
line = base.mark_line(color=colors["eco-turquiose"]).encode(
    y=alt.Y(
        "a:Q",
        sort=[],
        axis=alt.Axis(
            grid=True,
            title="",
            titleAnchor="start",
            labelColor=colors["eco-gray"],
            titleColor=colors["eco-gray"],
            tickColor=colors["eco-gray"],
            domainColor=colors["eco-gray"],
            gridColor=colors["eco-gray"],
            gridOpacity=0.1,
            titleFontSize=10,
            titleFontWeight="normal",
            ticks=False,
            labelAlign="left",
            labelBaseline="middle",
            labelPadding=-5,
            labelOffset=-10,
            titleX=22,
            titleY=7,
            titleBaseline="bottom",
            titleAngle=0,
            titleAlign="left",
            tickCount=7,
            format=".1f",
        ),
        scale=alt.Scale(domain=[-3.0, 1]),
    )
)
zeroLine=base.mark_line(color=colors['eco-gray'],opacity=0.5).encode(y='z:Q')
area = line.mark_area(opacity=0.2, color=colors["eco-turquiose"]).encode(
    y="bottom:Q", y2="top:Q"
)
bars = base.mark_bar(width=20, opacity=0.5,color=colors["eco-light-blue"]).encode(
    y=alt.Y("bin:Q", axis=None),
    x=alt.X(
        "x:N",
        sort=[],
        axis=alt.Axis(
            grid=False,
            titleAlign="right",
            titleAnchor="end",
            titleY=-15,
            title="",
            labelColor=colors["eco-gray"],
            titleColor=colors["eco-gray"],
            tickColor=colors["eco-gray"],
            domainColor=colors["eco-gray"],
            tickCount=7,
            orient="bottom",
            labelAngle=0,
        ),
        # scale=alt.Scale(domain=[1, 14], nice=False),
    ),
)
title = alt.TitleParams(
    "Change in productivity at different temperatures",
    subtitle=["Change in output per ha."],
    anchor="start",
    align="left",
    dx=5,
    dy=-5,
    fontSize=12,
    subtitleFontSize=11,
    subtitleFontStyle="italic",
)
layer1 = (
    (
        (zeroLine+area + line).properties(height=300, width=400)
        & bars.properties(height=50, width=400)
    )
    .configure_view(stroke=None)
    .properties(title=title)
)
layer1.save("visualisation/" + f + ".json")
layer1.save("visualisation/" + f + ".png")
open("README.md", "a").write(readme)
layer1

Fontconfig error: Cannot load default config file


alt.VConcatChart(...)

In [492]:
theme = "_dark"

layer1 = (
    (
        (zeroLine+area + line).properties(height=300, width=400)
        & bars.properties(height=50, width=400)
    )
    .configure_view(stroke=None)
    .properties(title=title)
)
layer1 = layer1.configure_axisYQuantitative(labelFontSize=12)
layer1 = layer1.configure_axisXQuantitative(labelFontSize=12)
layer1.config.font="Georgia"
layer1.config.background=colors["eco-background"]
layer1.config.view.stroke=None
layer1.title.fontSize = 14
layer1.title.subtitleFontSize = 12
layer1.title.dy -= 2
layer1.title.color = colors["eco-dot"]
layer1.title.subtitleColor = colors["eco-dot"]
layer1.save("visualisation/" + f + theme + ".json")
layer1.save("visualisation/" + f + theme + ".png")
readme = re.sub(f, f + theme, readme)
open("README.md", "a").write(readme)
layer1

Fontconfig error: Cannot load default config file


alt.VConcatChart(...)

# Fig 2

In [493]:
# df = pd.read_excel("raw/Book1.xlsx").dropna(axis=1, how="all")
# dfs = []
# for c in df.columns:
#     dx = df[[c]]
#     dx["x"] = dx[c].str.split(",").str[0]
#     dx["y"] = dx[c].str.split(",").str[1]
#     dx["year"] = c
#     dx = dx.drop(c, axis=1)
#     dx["x"] = dx["x"].astype(float).round(0)
#     dfs.append(dx)
# df = pd.concat(dfs,axis=1).dropna()
# df.to_excel('raw/fig2.xlsx')
df=pd.read_excel('raw/fig2.xlsx')
df['z']=0

In [494]:
f = "fig2_yield"
f2 = eco_git_path + f + ".csv"
df.to_csv("data/" + f + ".csv")
f += local_suffix
open("visualisation/" + f + ".html", "w").write(
    vega_embed.replace(
        "JSON_PATH", f2.replace("/data/", "/visualisation/").replace(".csv", ".json")
    )
)
if LOCAL:
    f2 = df
readme = "### " + f + '\n!["' + f + '"](visualisation/' + f + '.png "' + f + '")\n\n'
df.head()

,x,top,a,bottom,z
0,-5,0.049219,0.020500,-0.007443,0
1,-1,0.048935,0.020474,-0.007468,0
2,0,0.039613,0.009083,-0.020930,0
3,4,0.039585,0.009575,-0.020956,0
4,5,0.040073,0.019141,-0.002333,0


In [495]:
base = alt.Chart(f2).encode(
    x=alt.X(
        "x:N",
        sort=[],
        axis=alt.Axis(
            grid=False,
            titleAlign="right",
            titleAnchor="end",
            titleY=-15,
            title="Temperature (°C)",
            labelColor=colors["eco-gray"],
            titleColor=colors["eco-gray"],
            tickColor=colors["eco-gray"],
            domainColor=colors["eco-gray"],
            tickCount=7,
            orient="bottom",
            labelAngle=0,
        ),
        # scale=alt.Scale(domain=[1, 14], nice=False),
    )
)
line = base.mark_line(color=colors["eco-turquiose"]).encode(
    y=alt.Y(
        "a:Q",
        sort=[],
        axis=alt.Axis(
            grid=True,
            title="log-yield (bu per hectare)",
            titleAnchor="start",
            labelColor=colors["eco-gray"],
            titleColor=colors["eco-gray"],
            tickColor=colors["eco-gray"],
            domainColor=colors["eco-gray"],
            gridColor=colors["eco-gray"],
            gridOpacity=0.1,
            titleFontSize=10,
            titleFontWeight="normal",
            ticks=False,
            labelAlign="left",
            labelBaseline="middle",
            labelPadding=-5,
            labelOffset=-10,
            titleX=30,
            titleY=35,
            titleBaseline="bottom",
            titleAngle=0,
            titleAlign="left",
            tickCount=4,
            format=".2f",
        ),
#         scale=alt.Scale(domain=[-3.0, 1]),
    )
)
zeroLine=base.mark_line(color=colors['eco-gray'],opacity=0.5).encode(y='z:Q')
area = line.mark_area(opacity=0.2, color=colors["eco-turquiose"]).encode(
    y="bottom:Q", y2="top:Q"
)
title = alt.TitleParams(
    "Effects of temperature on wheat yields",
    subtitle=["Data from South Africa. Source: Shew et al. (2020)"],
    anchor="start",
    align="left",
    dx=5,
    dy=-5,
    fontSize=12,
    subtitleFontSize=11,
    subtitleFontStyle="italic",
)
layer1 = (
    (
        (zeroLine+area + line).properties(height=300, width=400)
    )
    .configure_view(stroke=None)
    .properties(title=title)
)
layer1.save("visualisation/" + f + ".json")
layer1.save("visualisation/" + f + ".png")
open("README.md", "a").write(readme)
layer1

Fontconfig error: Cannot load default config file


alt.LayerChart(...)

In [496]:
line.encoding.y.axis.titleFontSize = 12
line.encoding.y.axis.titleY += 5
line.encoding.y.axis.titleX += 3

theme = "_dark"
layer1 = (
    (
        (zeroLine+area + line).properties(height=300, width=400)
    )
    .configure_view(stroke=None)
    .properties(title=title)
)
layer1 = layer1.configure_axisYQuantitative(labelFontSize=12)
layer1 = layer1.configure_axisXQuantitative(labelFontSize=12)
layer1.config.font="Georgia"
layer1.config.background=colors["eco-background"]
layer1.config.view.stroke=None
layer1.title.fontSize = 14
layer1.title.subtitleFontSize = 12
layer1.title.dy -= 2
layer1.title.color = colors["eco-dot"]
layer1.title.subtitleColor = colors["eco-dot"]
layer1.save("visualisation/" + f + theme + ".json")
layer1.save("visualisation/" + f + theme + ".png")
readme = re.sub(f, f + theme, readme)
open("README.md", "a").write(readme)
layer1

Fontconfig error: Cannot load default config file


alt.LayerChart(...)

# Fig 3

In [497]:
df = pd.read_csv("raw/data_noaa.csv", skiprows=4)

In [498]:
f = "fig3_africa"
f3 = eco_git_path + f + ".csv"
df.to_csv("data/" + f + ".csv")
f += local_suffix
open("visualisation/" + f + ".html", "w").write(
    vega_embed.replace(
        "JSON_PATH", f3.replace("/data/", "/visualisation/").replace(".csv", ".json")
    )
)
if LOCAL:
    f3 = df
readme = "### " + f + '\n!["' + f + '"](visualisation/' + f + '.png "' + f + '")\n\n'
df.head()

,Year,Value
0,1910,-0.35
1,1911,-0.56
2,1912,-0.19
3,1913,-0.08
4,1914,0.00


In [ ]:
base = alt.Chart(f3).encode(
    x=alt.X(
        "Year:Q",
        sort=[],
        axis=alt.Axis(
            grid=False,
            titleAlign="center",
            titleAnchor="middle",
            title="",
            labelColor=colors["eco-gray"],
            titleColor=colors["eco-gray"],
            tickColor=colors["eco-gray"],
            domainColor=colors["eco-gray"],
            tickCount=7,
            orient="bottom",
            labelAngle=0,
            format=".0f",
        ),
        scale=alt.Scale(domain=[1896, 2025], nice=False),
    )
)
bars1 = (
    base.mark_bar(color=colors["eco-dot"])
    .encode(
        y=alt.Y(
            "Value:Q",
            sort=[],
            axis=alt.Axis(
                grid=True,
                title="°C",
                titleAnchor="start",
                labelColor=colors["eco-gray"],
                titleColor=colors["eco-gray"],
                tickColor=colors["eco-gray"],
                domainColor=colors["eco-gray"],
                gridColor=colors["eco-gray"],
                gridOpacity=0.1,
                titleFontSize=10,
                titleFontWeight="normal",
                ticks=False,
                labelAlign="left",
                labelBaseline="middle",
                labelPadding=-5,
                labelOffset=-10,
                titleX=22,
                titleY=7,
                titleBaseline="bottom",
                titleAngle=0,
                titleAlign="left",
                tickCount=7,
                format=".1f",
            ),
        )
    )
    .transform_filter("datum.Value>0")
)
bars2 = (
    base.mark_bar(color=colors["eco-light-blue"])
    .encode(y=alt.Y("Value:Q", sort=[]))
    .transform_filter("datum.Value<0")
)
axis1 = (
    alt.Chart(pd.DataFrame([{"x": 1895, "y": 0}, {"x": 2025, "y": 0}]))
    .mark_line(color=colors["eco-gray"])
    .encode(x=alt.X("x:Q", sort=[]), y="y:Q")
)
label = (
    alt.Chart(pd.DataFrame([{"x": 2025, "y": 0, "t": "20th century average"}]))
    .mark_text(dy=5, align="right", baseline="top", color=colors["eco-gray"])
    .encode(x="x:Q", y="y:Q", text="t:N")
)
title = alt.TitleParams(
    "Temperature anomaly in Africa",
    subtitle=[
        "January-December temperatures compared to the 20th century average. Source: NOAA"
    ],
    anchor="start",
    align="left",
    dx=5,
    dy=-5,
    fontSize=12,
    subtitleFontSize=11,
    subtitleFontStyle="italic",
)
layer1 = (
    ((bars1 + bars2 + axis1 + label).properties(height=300, width=400))
    .configure_view(stroke=None)
    .properties(title=title)
)
layer1.save("visualisation/" + f + ".json")
layer1.save("visualisation/" + f + ".png")
open("README.md", "a").write(readme)
layer1

In [ ]:
theme = "_dark"

bars1.encoding.y.axis.titleFontSize = 12
bars1.encoding.y.axis.titleY += 5
bars1.encoding.y.axis.titleX += 3
label.mark.fontSize = 12

layer1 = (
    ((bars1 + bars2 + axis1 + label).properties(height=300, width=400))
    .configure(font="Georgia", background=colors["eco-background"])
    .configure_view(stroke=None)
    .properties(title=title)
)
layer1 = layer1.configure_axisYQuantitative(labelFontSize=12)
layer1 = layer1.configure_axisXQuantitative(labelFontSize=12)
layer1.title.fontSize = 14
layer1.title.subtitleFontSize = 12
layer1.title.dy -= 2
layer1.title.color = colors["eco-dot"]
layer1.title.subtitleColor = colors["eco-dot"]
layer1.save("visualisation/" + f + theme + ".json")
layer1.save("visualisation/" + f + theme + ".png")
readme = re.sub(f, f + theme, readme)
open("README.md", "a").write(readme)
layer1

# Fig 4

In [ ]:
# https://impactlab.org/map/#usmeas=absolute&usyear=1981-2010&gmeas=absolute&gyear=1986-2005&tab=global
# https://climateknowledgeportal.worldbank.org/